# Healthgrades.com Review Web Scraper

By Jacob Hancock

### Import Required Packages

In [58]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [89]:
html_text = requests.get('https://www.healthgrades.com/family-practice-directory')
html_text

<Response [200]>

Response [200] means we have access to scrape the website.

In [140]:
html_text = requests.get('https://www.healthgrades.com/family-practice-directory').text
soup = BeautifulSoup(html_text, 'lxml')
doctors = soup.find_all('div', class_ = 'sd-provider-card')

Initialize dataframes 

In [141]:
Review_texts = []
Review_rating = []
Review_dates = []
Number_of_reviews = []
Physician_names = []
Physician_specialty = []
Physician_location = []
Physician_sex = []

In [148]:
for doctor in doctors:
    doctor_name = doctor.find('a', class_ = '_1qaR5')['aria-label']
    doctor_location = doctor.find('div', class_ = 'directory-card-location__office-loc').find_next('span').find_next('span').find_next('span').text
    link = 'https://www.healthgrades.com/' + doctor.find('a', class_ = '_1qaR5')['href']
    link_text = requests.get(link).text
    doctor_link = BeautifulSoup(link_text, 'lxml') 
    review_all = doctor_link.find_all('div', class_ = 'c-single-comment')
    num_reviews = len(review_all)
    doctors_specialty = doctor.find('div', class_ = 'provider-details__subtitle').find_next('span').text
    try: 
        doctors_sex = doctor_link.find('div', class_ = 'summary-column').find_next('span').find_next('span').find_next('span').text
    except:
        doctors_sex = doctor_link.find('div', class_ = 'summary-designated-header-image-container').find_next('span').find_next('span').find_next('span').text
    for review in review_all:
        Physician_sex.append(doctors_sex)
        Number_of_reviews.append(num_reviews)
        Physician_names.append(doctor_name)
        Physician_location.append(doctor_location)
        Physician_specialty.append(doctors_specialty)
        reviews = review.find('div', class_ = 'c-single-comment__comment').text
        Review_texts.append(reviews)
        review_dates = review.find('div', class_ = 'c-single-comment__commenter-info').find_next('span').find_next('span').text
        Review_dates.append(review_dates)
        review_rating = review.find('span', class_ = 's6RLV')["aria-label"]
        Review_rating.append(review_rating)

In [145]:
Review_Dates = pd.Series(Review_dates, dtype = "string", name = "Review Date")
Review_Texts = pd.Series(Review_texts, dtype = "string", name = "Review Text")
Physician_Names = pd.Series(Physician_names, dtype = "string", name = "Physician Name")
Physician_Location = pd.Series(Physician_location, dtype = "string", name = "Physician Location")
Physician_Specialty = pd.Series(Physician_specialty, dtype = "string", name = "Physician Specialty")
Review_Rating = pd.Series(Review_rating, dtype = "string", name = "Review Rating")
Physician_Sex = pd.Series(Physician_sex, dtype = "string", name = "Physician Sex")
Number_of_Reviews = pd.Series(Number_of_reviews, dtype = "string", name = "Number of Reviews")

In [146]:
Example_reviews = pd.concat([Physician_Names, Physician_Sex, Physician_Location, Physician_Specialty, Number_of_Reviews, Review_Dates, Review_Texts, Review_Rating], axis = 1)
Example_reviews

,Physician Name,Physician Sex,Physician Location,Physician Specialty,Number of Reviews,Review Date,Review Text,Review Rating
0,"Dr. Marc Harwood, MD",Male,"Philadelphia, PA 19107",Sports Medicine,20,"Jan 15, 2021",Dr Harwood was fully understanding of the issu...,Rated 5 out of 5
1,"Dr. Marc Harwood, MD",Male,"Philadelphia, PA 19107",Sports Medicine,20,"Jan 08, 2021",I was having elbow pain for several months and...,Rated 5 out of 5
2,"Dr. Marc Harwood, MD",Male,"Philadelphia, PA 19107",Sports Medicine,20,"Oct 11, 2020","Dr Harwood listened to my concerns, explained ...",Rated 5 out of 5
3,"Dr. Marc Harwood, MD",Male,"Philadelphia, PA 19107",Sports Medicine,20,"Oct 10, 2020",Minimal wait time . Great Doctor ! Clean envi...,Rated 5 out of 5
4,"Dr. Marc Harwood, MD",Male,"Philadelphia, PA 19107",Sports Medicine,20,"Sep 05, 2020","Fantastic, conscious doctor. Dr. Harwood take...",Rated 5 out of 5
...,...,...,...,...,...,...,...,...
547,"Dr. Robert Frable, DO",Male,"Crawfordville, FL 32327",Family Medicine,20,"Feb 22, 2019",Dr. Frable is an exceptionally able physician ...,Rated 5 out of 5
548,"Dr. Robert Frable, DO",Male,"Crawfordville, FL 32327",Family Medicine,20,"Jan 16, 2019","Dr. Frable is very knowledgeable, caring, and ...",Rated 5 out of 5
549,"Dr. Robert Frable, DO",Male,"Crawfordville, FL 32327",Family Medicine,20,"Oct 27, 2018",I have trusted Dr. Frable for many years and h...,Rated 5 out of 5
550,"Dr. Robert Frable, DO",Male,"Crawfordville, FL 32327",Family Medicine,20,"Oct 22, 2018","Great office, friendly staff, clean and effici...",Rated 5 out of 5


In [132]:
Example_reviews['Physician Name'].unique()

<StringArray>
[        'Dr. Marc Harwood, MD',           'Dr. Paula Hall, MD',
      'Dr. Andre Duplantis, MD',        'Dr. Lauren Heavin, MD',
         'Dr. Ted Hudspeth, MD',           'Dr. James Wade, MD',
       'Dr. Douglas Cutter, MD',      'Dr. Valerie Goodman, DO',
  'Dr. Susan Dallas-Feeney, DO', 'Dr. Christopher Holloway, MD',
             'Dr. Joe Dunn, MD',      'Dr. Gena Pennington, MD',
      'Dr. Patrick Mulcahy, DO',          'Dr. Bogale Jima, MD',
         'Dr. Ahmad Aslami, DO',         'Dr. Daniel Jasko, MD',
     'Dr. Ambrose Peterman, DO',       'Dr. Maurice Oakley, MD',
         'Dr. Philip Weiss, MD',       'Dr. Richard Putnam, MD',
        'Dr. Shawn Wallace, MD',           'Dr. Mark James, MD',
       'Dr. Philippe Chain, MD',         'Dr. Eugene Eskow, MD',
            'Dr. Mark Drew, MD',    'Dr. Shailaja Malhotra, MD',
         'Dr. Keith Elkins, MD',           'Dr. Lori Spoor, DO',
        'Dr. David Klibert, MD',    'Dr. Christine Martino, DO',
      'Dr. 

### Convert to Excel Sheet

In [123]:
Example_reviews.to_csv(r'C:\Users\Jake\Desktop\UMass Amherst\Medical Review Research\Example_reviews.csv', index = False, header=True)

## Scrape from Individual Page

In [133]:
html_text = requests.get('https://www.healthgrades.com/physician/dr-paul-baecher-xcfrf').text
soup = BeautifulSoup(html_text, 'lxml')

#### Review Date

In [134]:
review_date = soup.find('div', class_ = 'c-single-comment__commenter-info').find_next('span').find_next('span').text
review_date

'Jul 14, 2017 '

#### Physician's Name

In [17]:
doctors_name = soup.find_all('li', class_ = 'breadcrumb-link-container')[3].find('a', class_ = 'breadcrumb-current-page breadcrumb-link').text
doctors_name

'Dr. Paul Baecher, MD'

#### Physician's Sex

- Since there are 2 different styles of pages, both need to be accounted for:

First type:

In [135]:
html_text = requests.get('https://www.healthgrades.com/physician/dr-lauren-heavin-xymqgpy').text
soup = BeautifulSoup(html_text, 'lxml')

Second type:

In [136]:
html_text = requests.get('https://www.healthgrades.com/physician/dr-paul-baecher-xcfrf').text
soup = BeautifulSoup(html_text, 'lxml')

Works for either page:

In [137]:
try: 
    doctors_sex = soup.find('div', class_ = 'summary-column').find_next('span').find_next('span').find_next('span').text
    print(doctors_sex)
except:
    doctors_sex = soup.find('div', class_ = 'summary-designated-header-image-container').find_next('span').find_next('span').find_next('span').text
    print(doctors_sex)

Male


#### Review Text

In [13]:
review = soup.find('div', class_ = 'c-single-comment__comment').text
review

'Very caring and not afraid to refer you to a specialist.\nNice caring doctor and man, listens well.\n'

#### Review Rating

In [14]:
review_rating = soup.find('span', class_ = 's6RLV')["aria-label"]
review_rating

'Rated 4 out of 5'